# To make a network of the organisms in my garden

In [3]:
import sys
print(sys.version)

#Python 3.7.4 (default, Aug 9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
#pygbif 0.3.0

3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]


In [4]:
import pandas as pd
#import re
#import matplotlib.pyplot as plt
#from pygbif import species
#from pygbif import occurrences as occ
import sqlite3
from sqlite3 import Error
#import pydot
#import math
import csv

### Setting up some parameters

In [5]:
# If you want to exclude an interaction add it here
#interactionsToExclude = "visitsFlowersOf" #If you're not intersted in pollinators this will remove most of them
interactionsToExclude = "interactsWith"    #Many interactions in GloBI do not specify a specific interaction. You can get a cleaner netwot by excluding them
#interactionsToExclude = ""

In [6]:
## A script to create globi.db is here https://github.com/AgentschapPlantentuinMeise/createGlobiDB
## Define the place to find the interaction data
## This just avoids having to rely on the GloBI API, but it also mean that a stable citable version of GloBI can be used.
globiDB = r"..\..\createGlobiDB\globi.db"

In [7]:
# import a file of observations from iNaturalist
taxaList = {} # set up a dictionary of names


#iNat = pd.read_csv ('observations-125623.csv')

#for index, row in iNat.iterrows():
#    taxaList[row['scientific_name']] = row['iconic_taxon_name']

gbif = pd.read_csv('0146461-200613084148143.csv',sep='\t')

for index, row in gbif.iterrows():
    taxaList[row['species']] = row['kingdom']

In [8]:
# Add humans to the network
taxaList['Homo sapiens'] = 'Animalia'

In [9]:
#Create a connection to the SGLite version of GloBI
try:
    connGlobi = sqlite3.connect(globiDB)
except Error as e:
    print(e)

## Get the primary interation data for the species in question

This is limited to the rank of species because otherwise the network can get very large with rather irrelivent higher taxa.
However, if no interactions are found at the species level it might be worth removing this restriction.

In [10]:
def fetchInteractionForTaxon(taxon, interactDataTaxon):
    
    curGlobi = connGlobi.cursor()
    if interactionsToExclude == "":
        curGlobi.execute("SELECT * from globi  WHERE targetTaxonSpeciesName = ?;", \
                     (taxon,))
    else:
        curGlobi.execute("SELECT * from globi  WHERE targetTaxonSpeciesName = ? and interactionTypeName NOT IN (?);", \
                     (taxon,interactionsToExclude,))
        interactDataTaxon.extend(curGlobi.fetchall())
    
    curGlobi = connGlobi.cursor()
    if interactionsToExclude == "":
        curGlobi.execute("SELECT * from globi  WHERE sourceTaxonSpeciesName = ?;", \
                     (taxon,))
    else:
        curGlobi.execute("SELECT * from globi  WHERE sourceTaxonSpeciesName = ? and interactionTypeName NOT IN (?);", \
                     (taxon,interactionsToExclude,))
        interactDataTaxon.extend(curGlobi.fetchall())

In [11]:
interactDataTaxon = []

for animal in taxaList:
    print(animal)
    fetchInteractionForTaxon(animal,interactDataTaxon)
    

Juncus effusus
Isolepis setacea
Tropaeolum majus
Populus jackii
Tripleurospermum maritimum


KeyboardInterrupt: 

In [12]:
print("Number of interactions found in gloBI ", len(interactDataTaxon))

Number of interactions found in gloBI  878


In [13]:
# Convert to a Pandas dataframe
interactDataTaxon = pd.DataFrame(interactDataTaxon)

In [14]:
# Add column names
interactDataTaxon
interactDataTaxon.columns = ['sourceTaxonId', \
                                'sourceTaxonIds','sourceTaxonName','sourceTaxonRank','sourceTaxonPathNames', \
                                'sourceTaxonPathIds','sourceTaxonPathRankNames','sourceTaxonSpeciesName','sourceTaxonSpeciesId',\
                                'sourceTaxonGenusName','sourceTaxonGenusId','sourceTaxonFamilyName','sourceTaxonFamilyId',\
                                'sourceTaxonOrderName','sourceTaxonOrderId','sourceTaxonClassName','sourceTaxonClassId',\
                                'sourceTaxonPhylumName','sourceTaxonPhylumId','sourceTaxonKingdomName','sourceTaxonKingdomId',\
                                'sourceId','sourceOccurrenceId','sourceCatalogNumber','sourceBasisOfRecordId',\
                                'sourceBasisOfRecordName','sourceLifeStageId','sourceLifeStageName','sourceBodyPartId',\
                                'sourceBodyPartName','sourcePhysiologicalStateId','sourcePhysiologicalStateName', \
                                'sourceSexId', 'sourceSexName','interactionTypeName',\
                                'interactionTypeId','targetTaxonId','targetTaxonIds','targetTaxonName',\
                                'targetTaxonRank','targetTaxonPathNames','targetTaxonPathIds','targetTaxonPathRankNames',\
                                'targetTaxonSpeciesName','targetTaxonSpeciesId','targetTaxonGenusName','targetTaxonGenusId',\
                                'targetTaxonFamilyName','targetTaxonFamilyId','targetTaxonOrderName','targetTaxonOrderId',\
                                'targetTaxonClassName','targetTaxonClassId','targetTaxonPhylumName','targetTaxonPhylumId',\
                                'targetTaxonKingdomName','targetTaxonKingdomId','targetId','targetOccurrenceId',\
                                'targetCatalogNumber','targetBasisOfRecordId','targetBasisOfRecordName','targetLifeStageId',\
                                'targetLifeStageName','targetBodyPartId','targetBodyPartName','targetPhysiologicalStateId',\
                                'targetPhysiologicalStateName', 'targetSexId', 'targetSexName',\
                                'decimalLatitude','decimalLongitude','localityId',\
                                'localityName','eventDateUnixEpoch','argumentTypeId','referenceCitation',\
                                'referenceDoi','referenceUrl','sourceCitation','sourceNamespace',\
                                'sourceArchiveURI','sourceDOI','sourceLastSeenAtUnixEpoch','combo']

## Get a list of all the primary interacting species

In [15]:
# How many different sort of interaction do I have left?
# Checking out all the interaction types
interactDataTaxon.groupby(interactDataTaxon['interactionTypeName']).size().sort_values(ascending = False)

interactionTypeName
hasHost               523
eats                  270
visitsFlowersOf        38
parasiteOf             30
pollinates             13
hasDispersalVector      3
pathogenOf              1
dtype: int64

## Drawing a network of the interactions

### Put the network together using the networkx package

In [19]:
#networkx seems to be a leading network tool in Python
import networkx as nx
import matplotlib.pyplot as plt

try:
    import pygraphviz
    from networkx.drawing.nx_agraph import write_dot
    print("using package pygraphviz")
except ImportError:
    try:
        import pydot
        from networkx.drawing.nx_pydot import write_dot
        print("using package pydot")
    except ImportError:
        print()
        print("Both pygraphviz and pydot were not found ")
        print("see  https://networkx.github.io/documentation/latest/reference/drawing.html")
        print()

using package pydot


In [20]:
# Create graphic object
G = nx.DiGraph()

## A quick look at the interaction data to see if it is what is expected

In [21]:
#cleanInteractDataTaxon.loc[(cleanInteractDataTaxon["sourceTaxonName"] == 'Apis mellifera') & (cleanInteractDataTaxon["targetTaxonName"] == 'Procyon lotor')]
interactDataTaxon.loc[(interactDataTaxon["sourceTaxonSpeciesName"] == 'Nyctereutes procyonoides')]

,sourceTaxonId,sourceTaxonIds,sourceTaxonName,sourceTaxonRank,sourceTaxonPathNames,sourceTaxonPathIds,sourceTaxonPathRankNames,sourceTaxonSpeciesName,sourceTaxonSpeciesId,sourceTaxonGenusName,...,argumentTypeId,referenceCitation,referenceDoi,referenceUrl,sourceCitation,sourceNamespace,sourceArchiveURI,sourceDOI,sourceLastSeenAtUnixEpoch,combo


### Take out any interactions that are for the same pair of species and interaction type

In [25]:
interactDataTaxonNoDups = interactDataTaxon.drop_duplicates(subset=['sourceTaxonSpeciesName', 'targetTaxonSpeciesName', 'interactionTypeName'])

In [26]:
len(interactDataTaxonNoDups)

192

### Add the nodes to the graph

In [27]:
# Kingdom is added because it is useful for visualization in Gehpi

for key in taxaList:
    G.add_node(key, kingdom=taxaList[key])

### Add edges to the graph

In [28]:
# iterate over the interacting species
# then add the edge if the target species exists.

#for index, row in cleanInteractDataTaxon.iterrows():
    
taxaList2 = taxaList

for key in taxaList:
    # loop over all the taxa finding if any of them are mentioned in the sourceTaxonName field
    for edge in interactDataTaxonNoDups.iterrows():
        if key == edge[1]['sourceTaxonSpeciesName']:
            #print('B: ', edge[1]['sourceTaxonName'], edge[1]['targetTaxonName'])
            # Some of the target species will not be in GBIF of in the country, so only add an edge where they are.
            for key2 in taxaList2:
                #print('E: ', row2['species'], edge[1]['targetTaxonName'])
                if key2 == edge[1]['targetTaxonSpeciesName']: 
                    print('C: ', key, key2, edge[1]['interactionTypeName'])
                    G.add_edge(key, key2, label = edge[1]['interactionTypeName'])


C:  Juncus effusus Fringilla coelebs hasDispersalVector
C:  Juncus effusus Sylvia atricapilla hasDispersalVector
C:  Bombus terrestris Tropaeolum majus visitsFlowersOf
C:  Pieris napi Tropaeolum majus eats
C:  Vanessa cardui Tropaeolum majus eats
C:  Vanessa cardui Tropaeolum majus visitsFlowersOf
C:  Apis mellifera Tropaeolum majus eats
C:  Apis mellifera Tropaeolum majus visitsFlowersOf
C:  Aphis fabae Tropaeolum majus eats


In [29]:
print("Number of nodes = {0}".format(G.number_of_nodes()))
print("Number of edges = {0}".format(G.number_of_edges()))

Number of nodes = 1042
Number of edges = 7


## Remove any nodes that have no edges.


In [30]:
#for n in G.nodes:
#    if G.degree(n) == 0:
#        G.remove_node(n)
        
G.remove_nodes_from(list(nx.isolates(G)))

In [31]:
len(G.nodes)

9

## Remove any selfloop edges

In [32]:
for e in G.selfloop_edges(data=False):
    G.remove_edge(e[0],e[1])

### For import into Gephi: A nodes and edges file has been replace by the .dot file format

In [59]:
write_dot(G, "..\\docs\\"+"network"+".dot")